<a href="https://colab.research.google.com/github/bzhao10/cs598_dlh_final_project/blob/main/CS_598_Final_pj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS 598 Final Project

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
import pandas as pd

notes=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/noteevents.csv")

diag_icd=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/diagnose_icd_noteevents_dataset.csv")
admin_icd=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/admission_id_diagnose_icd_code.csv")

In [6]:
admin_icd.head()

,HADM_ID,icd_codes
0,100001,"25013,25043,25053,25063,25083,3371,36201,40390..."
1,100009,"25000,2720,27800,2859,4019,4111,41401,4142,414..."
2,100018,"2254,25040,25060,25070,2761,27800,3363,3572,40..."
3,100034,"25000,2724,4019,412,4139,41401,6961,V1582"
4,100038,"25080,2720,27650,2859,4019,41400,4589,5849,786..."


In [53]:
def icd_percentage(admin_icd,s):
  res=0
  for i in range(len(admin_icd)):
    if s in admin_icd.loc[i,'icd_codes']:
      res=res+1
  return res/len(admin_icd)
icd_percentage(admin_icd,'272')

0.4361348695836573

In [41]:
icd_list=[]
for i in range(len(admin_icd)):
  icd_list=icd_list+admin_icd.loc[i,'icd_codes'].split(',')
print(len(set(icd_list))#4097

AttributeError: ignored

In [50]:
icd_list=set(icd_list)
icd_list_roll=[]
for i in icd_list:
  icd_list_roll.append(i[0:3])
print(len(set(icd_list_roll)))#780

780


In [57]:
import time
import numpy as np
import string
from nltk import word_tokenize


# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT


#input
# corpus_df: a dataframe returned by `load_dataset`
#output
# corpus_processed: a list of articles where each article is broke into a list of words 
def preprocess_dataset(df):    
    ''' Preprocess the text data. And return a list of articles. '''
    corpus_processed = []
    corpus_all=[]
    df.TEXT = df.TEXT.str.replace('\n', '')  # remove newline
    df.TEXT = df.TEXT.str.replace('\r', '')  # carriage returns
    """
    TODO: 1. remove punc;
          2. remove numbers.
          
    HINT: consider using `string.punctuation`, `str.maketrans`, and `str.translate`.
    """
    #df.TEXT=df.TEXT.str.replace(string.punctuation, '')

    table = str.maketrans('', '', string.punctuation)
    df.TEXT = [w.translate(table) for w in df.TEXT]
    table2 = str.maketrans('', '', '1234567890')
    df.TEXT = [w.translate(table2) for w in df.TEXT]    
    df.TEXT = df.TEXT.str.lower()  # convert to lower case
    
    # tokenize
    for note in df.TEXT.values:
        note_tokenized = word_tokenize(note)
        corpus_processed.append(note_tokenized)
        corpus_all=corpus_all+note_tokenized
    return corpus_processed,corpus_all

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [60]:
notes_preprocessed,notes_all=preprocess_dataset(notes)

In [67]:
tmp=pd.value_counts(notes_all)
word_mistake=tmp[tmp<5].index.tolist()


In [69]:
word_dic=tmp[tmp>=5].index.tolist()

['the',
 'and',
 'of',
 'to',
 'was',
 'with',
 'a',
 'on',
 'in',
 'mg',
 'is',
 'for',
 'no',
 'name',
 'tablet',
 'po',
 'at',
 'patient',
 'he',
 'blood',
 'or',
 'as',
 'left',
 'she',
 'sig',
 'one',
 'normal',
 'hospital',
 'his',
 'last',
 'am',
 'valve',
 'day',
 'pm',
 'her',
 'were',
 'had',
 'right',
 'not',
 'by',
 'there',
 'be',
 'are',
 'daily',
 'namepattern',
 'date',
 'an',
 'this',
 'discharge',
 'first',
 'from',
 'pain',
 'that',
 'aortic',
 'has',
 'you',
 'admission',
 'ventricular',
 'history',
 'have',
 'your',
 'which',
 'pt',
 'but',
 'ct',
 'seen',
 'hours',
 'status',
 'stitle',
 'times',
 'mild',
 'chest',
 'mitral',
 'tablets',
 'md',
 'refills',
 'well',
 'present',
 'leaflets',
 'sinus',
 'needed',
 'known',
 'medications',
 'capsule',
 'every',
 'up',
 'given',
 'artery',
 'x',
 'wall',
 'un',
 'also',
 'two',
 'prior',
 'dr',
 'release',
 'disease',
 'care',
 'after',
 'medical',
 'any',
 'started',
 'per',
 'plt',
 'size',
 'stable',
 'ml',
 'time',